In [2]:
import os
import zipfile
import random
import tensorflow as tf
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from shutil import copyfile

In [ ]:
local_zip = 'C:/Users/LALITH/Downloads/cats-and-dogs.zip'
zip_ref   = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('/tmp')
zip_ref.close()

In [3]:
print(len(os.listdir('C:/Users/LALITH/Downloads/PetImages/Cat/')))
print(len(os.listdir('C:/Users/LALITH/Downloads/PetImages/Dog/')))

12501
12501


In [5]:
try:
    os.mkdir('C:/Users/LALITH/Downloads/cats-vs-dogs')
    os.mkdir('C:/Users/LALITH/Downloads/cats-vs-dogs/training')
    os.mkdir('C:/Users/LALITH/Downloads/cats-vs-dogs/testing')
    os.mkdir('C:/Users/LALITH/Downloads/cats-vs-dogs/training/cats')
    os.mkdir('C:/Users/LALITH/Downloads/cats-vs-dogs/training/dogs')
    os.mkdir('C:/Users/LALITH/Downloads/cats-vs-dogs/testing/cats')
    os.mkdir('C:/Users/LALITH/Downloads/cats-vs-dogs/testing/dogs')
except OSError:
    pass

In [11]:
def split_data(SOURCE, TRAINING, TESTING, SPLIT_SIZE):
    files = []
    for filename in os.listdir(SOURCE):
        file = SOURCE + filename
        if os.path.getsize(file) > 0:
            files.append(filename)
        else:
            print(filename + " is zero length, so ignoring.")

    training_length = int(len(files) * SPLIT_SIZE)
    testing_length = int(len(files) - training_length)
    shuffled_set = random.sample(files, len(files))
    training_set = shuffled_set[0:training_length]
    testing_set = shuffled_set[-testing_length:]

    for filename in training_set:
        this_file = SOURCE + filename
        destination = TRAINING + filename
        copyfile(this_file, destination)

    for filename in testing_set:
        this_file = SOURCE + filename
        destination = TESTING + filename
        copyfile(this_file, destination)


CAT_SOURCE_DIR = "C:/Users/LALITH/Downloads/PetImages/Cat/"
TRAINING_CATS_DIR = "C:/Users/LALITH/Downloads/cats-vs-dogs/training/cats/"
TESTING_CATS_DIR = "C:/Users/LALITH/Downloads/cats-vs-dogs/testing/cats/"
DOG_SOURCE_DIR = "C:/Users/LALITH/Downloads/PetImages/Dog/"
TRAINING_DOGS_DIR = "C:/Users/LALITH/Downloads/cats-vs-dogs/training/dogs/"
TESTING_DOGS_DIR = "C:/Users/LALITH/Downloads/cats-vs-dogs/testing/dogs/"

split_size = .9
split_data(CAT_SOURCE_DIR, TRAINING_CATS_DIR, TESTING_CATS_DIR, split_size)
split_data(DOG_SOURCE_DIR, TRAINING_DOGS_DIR, TESTING_DOGS_DIR, split_size)

666.jpg is zero length, so ignoring.
11702.jpg is zero length, so ignoring.


In [4]:
print(len(os.listdir('C:/Users/LALITH/Downloads/cats-vs-dogs/training/cats/')))
print(len(os.listdir('C:/Users/LALITH/Downloads/cats-vs-dogs/training/dogs/')))
print(len(os.listdir('C:/Users/LALITH/Downloads/cats-vs-dogs/testing/cats/')))
print(len(os.listdir('C:/Users/LALITH/Downloads/cats-vs-dogs/testing/dogs/')))

11250
11250
1250
1250


In [7]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(16, (3,3), activation='relu', input_shape=(150,150,3)),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid'),
    ])

In [8]:
model.compile(optimizer=RMSprop(lr=0.001), loss='binary_crossentropy', metrics=['acc'])

In [9]:
TRAINING_DIR = "C:/Users/LALITH/Downloads/cats-vs-dogs/training/"
train_datagen = ImageDataGenerator(rescale = 1.0/255)
train_generator = train_datagen.flow_from_directory(TRAINING_DIR, 
                                                    batch_size = 100,
                                                    class_mode ='binary',
                                                    target_size = (150,150))

VALIDATION_DIR = "C:/Users/LALITH/Downloads/cats-vs-dogs/testing/"
validation_datagen = ImageDataGenerator(rescale=1.0/255.)
validation_generator = validation_datagen.flow_from_directory(VALIDATION_DIR,
                                                              batch_size=100,
                                                              class_mode='binary',
                                                              target_size=(150, 150))

Found 22498 images belonging to 2 classes.
Found 2500 images belonging to 2 classes.


In [10]:
history = model.fit_generator(train_generator,
                             epochs = 50,
                             verbose = 1,
                             validation_data = validation_generator)

Epoch 1/50
 32/225 [===>..........................] - ETA: 10:36 - loss: 1.1243 - acc: 0.5334

C:\Users\LALITH\Anaconda3\lib\site-packages\PIL\TiffImagePlugin.py:763: UserWarning: Possibly corrupt EXIF data.  Expecting to read 18350080 bytes but only got 0. Skipping tag 0
  " Skipping tag %s" % (size, len(data), tag))
C:\Users\LALITH\Anaconda3\lib\site-packages\PIL\TiffImagePlugin.py:780: UserWarning: Corrupt EXIF data.  Expecting to read 12 bytes but only got 6. 
  warnings.warn(str(msg))


 46/225 [=====>........................] - ETA: 9:49 - loss: 0.9988 - acc: 0.5402

C:\Users\LALITH\Anaconda3\lib\site-packages\PIL\TiffImagePlugin.py:763: UserWarning: Possibly corrupt EXIF data.  Expecting to read 80000 bytes but only got 0. Skipping tag 64640
  " Skipping tag %s" % (size, len(data), tag))
C:\Users\LALITH\Anaconda3\lib\site-packages\PIL\TiffImagePlugin.py:763: UserWarning: Possibly corrupt EXIF data.  Expecting to read 65536 bytes but only got 0. Skipping tag 3
  " Skipping tag %s" % (size, len(data), tag))
C:\Users\LALITH\Anaconda3\lib\site-packages\PIL\TiffImagePlugin.py:763: UserWarning: Possibly corrupt EXIF data.  Expecting to read 307363840 bytes but only got 0. Skipping tag 5
  " Skipping tag %s" % (size, len(data), tag))
C:\Users\LALITH\Anaconda3\lib\site-packages\PIL\TiffImagePlugin.py:763: UserWarning: Possibly corrupt EXIF data.  Expecting to read 307888128 bytes but only got 0. Skipping tag 5
  " Skipping tag %s" % (size, len(data), tag))
C:\Users\LALITH\Anaconda3\lib\site-packages\PIL\TiffImagePlugin.py:763: UserWarning: Possibly corrup

 75/225 [=========>....................] - ETA: 8:15 - loss: 0.8723 - acc: 0.5705

...

C:\Users\LALITH\Anaconda3\lib\site-packages\PIL\TiffImagePlugin.py:763: UserWarning: Possibly corrupt EXIF data.  Expecting to read 6553600 bytes but only got 0. Skipping tag 49
  " Skipping tag %s" % (size, len(data), tag))
C:\Users\LALITH\Anaconda3\lib\site-packages\PIL\TiffImagePlugin.py:763: UserWarning: Possibly corrupt EXIF data.  Expecting to read 1050744 bytes but only got 4951. Skipping tag 51
  " Skipping tag %s" % (size, len(data), tag))
C:\Users\LALITH\Anaconda3\lib\site-packages\PIL\TiffImagePlugin.py:763: UserWarning: Possibly corrupt EXIF data.  Expecting to read 293339136 bytes but only got 0. Skipping tag 5
  " Skipping tag %s" % (size, len(data), tag))
C:\Users\LALITH\Anaconda3\lib\site-packages\PIL\TiffImagePlugin.py:763: UserWarning: Possibly corrupt EXIF data.  Expecting to read 293863424 bytes but only got 0. Skipping tag 5
  " Skipping tag %s" % (size, len(data), tag))
C:\Users\LALITH\Anaconda3\lib\site-packages\PIL\TiffImagePlugin.py:763: UserWarning: Possibly c

224/225 [============================>.] - ETA: 3s - loss: 0.6937 - acc: 0.6414

C:\Users\LALITH\Anaconda3\lib\site-packages\PIL\TiffImagePlugin.py:763: UserWarning: Possibly corrupt EXIF data.  Expecting to read 32 bytes but only got 0. Skipping tag 270
  " Skipping tag %s" % (size, len(data), tag))
C:\Users\LALITH\Anaconda3\lib\site-packages\PIL\TiffImagePlugin.py:763: UserWarning: Possibly corrupt EXIF data.  Expecting to read 5 bytes but only got 0. Skipping tag 271
  " Skipping tag %s" % (size, len(data), tag))
C:\Users\LALITH\Anaconda3\lib\site-packages\PIL\TiffImagePlugin.py:763: UserWarning: Possibly corrupt EXIF data.  Expecting to read 8 bytes but only got 0. Skipping tag 272
  " Skipping tag %s" % (size, len(data), tag))
C:\Users\LALITH\Anaconda3\lib\site-packages\PIL\TiffImagePlugin.py:763: UserWarning: Possibly corrupt EXIF data.  Expecting to read 8 bytes but only got 0. Skipping tag 282
  " Skipping tag %s" % (size, len(data), tag))
C:\Users\LALITH\Anaconda3\lib\site-packages\PIL\TiffImagePlugin.py:763: UserWarning: Possibly corrupt EXIF data.  Expec

225/225 [==============================] - 796s 4s/step - loss: 0.6931 - acc: 0.6419 - val_loss: 0.5179 - val_acc: 0.7504
Epoch 2/50
225/225 [==============================] - 980s 4s/step - loss: 0.5128 - acc: 0.7438 - val_loss: 0.4525 - val_acc: 0.7924
Epoch 3/50
225/225 [==============================] - 901s 4s/step - loss: 0.4434 - acc: 0.7950 - val_loss: 0.4280 - val_acc: 0.8016
Epoch 4/50
225/225 [==============================] - 806s 4s/step - loss: 0.3821 - acc: 0.8271 - val_loss: 0.4296 - val_acc: 0.8080
Epoch 5/50
225/225 [==============================] - 831s 4s/step - loss: 0.3256 - acc: 0.8553 - val_loss: 0.3916 - val_acc: 0.8224
Epoch 6/50
225/225 [==============================] - 781s 3s/step - loss: 0.2638 - acc: 0.8876 - val_loss: 0.4239 - val_acc: 0.8304
Epoch 7/50
225/225 [==============================] - 792s 4s/step - loss: 0.1984 - acc: 0.9193 - val_loss: 0.4471 - val_acc: 0.8292
Epoch 8/50
225/225 [==============================] - 750s 3s/step - loss: 0.138

KeyboardInterrupt: 

In [11]:
%matplotlib inline

import matplotlib.image  as mpimg
import matplotlib.pyplot as plt

#-----------------------------------------------------------
# Retrieve a list of list results on training and test data
# sets for each training epoch
#-----------------------------------------------------------
acc=history.history['acc']
val_acc=history.history['val_acc']
loss=history.history['loss']
val_loss=history.history['val_loss']

epochs=range(len(acc)) # Get number of epochs

#------------------------------------------------
# Plot training and validation accuracy per epoch
#------------------------------------------------
plt.plot(epochs, acc, 'r', "Training Accuracy")
plt.plot(epochs, val_acc, 'b', "Validation Accuracy")
plt.title('Training and validation accuracy')
plt.figure()

#------------------------------------------------
# Plot training and validation loss per epoch
#------------------------------------------------
plt.plot(epochs, loss, 'r', "Training Loss")
plt.plot(epochs, val_loss, 'b', "Validation Loss")
plt.figure()

NameError: name 'history' is not defined